# **Best city to live**

# Machine Learning

## Libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pycaret.regression import load_model, predict_model

## Reading the preprocessed datasets:

In [2]:
#city_df = pd.read_csv("../clean_data/city_df.csv", encoding='latin-1')
city_df = pd.read_csv("../clean_data/city_df.csv", encoding='latin-1')

In [3]:
city_df.columns

Index(['City', 'Country', 'Continent', 'Housing', 'Cost of Living', 'Startups',
       'Venture Capital', 'Travel Connectivity', 'Commute', 'Business Freedom',
       'Safety', 'Healthcare', 'Education', 'Environmental Quality', 'Economy',
       'Taxation', 'Internet Access', 'Leisure & Culture', 'Tolerance',
       'Outdoors', 'AQI Value', 'AQI Category', 'lat', 'lng'],
      dtype='object')

In [4]:
# Creo una seleccion de variables que quiero utilizar:

df = city_df[['City', 'Cost of Living','Travel Connectivity','Safety', 'Healthcare', 'Education', 'Environmental Quality', 'Economy',
       'Leisure & Culture']]

In [5]:
def fritas(df):
    df_encoded = df.copy()
    object_columns = df_encoded.select_dtypes(include=["object"]).columns
    encoder_info = []

    for column in object_columns:
        df_encoded[column] = pd.Categorical(df_encoded[column])
        df_encoded[column] = df_encoded[column].cat.codes
        df_encoded[column] = df_encoded[column].replace(-1, np.nan)
        
        encoder_info.append({
            "column": column,
            "labels": list(df[column].astype('category').cat.categories),
            "codes": list(range(len(df[column].astype('category').cat.categories))),
        })

    return df_encoded, encoder_info

df_encoded, encoder_info = fritas(df)

In [7]:
# Guardar encoder_info en un archivo JSON
with open('encoder_info.json', 'w') as file:
    json.dump(encoder_info, file)

In [8]:
from pycaret.regression import *

df_encoded, encoder_info = fritas(df)

# Configuración del entorno de PyCaret
regression_setup = setup(data=df_encoded, target='City')

# Comparar modelos y elegir el mejor
best_model = compare_models()

# Entrenar el mejor modelo
final_model = finalize_model(best_model)


,Description,Value
0,Session id,7640
1,Target,City
2,Target type,Regression
3,Original data shape,"(265, 9)"
4,Transformed data shape,"(265, 9)"
5,Transformed train set shape,"(185, 9)"
6,Transformed test set shape,"(80, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,63.5633,5615.5866,74.1589,-0.0647,0.9262,1.7823,0.0070
en,Elastic Net,64.2186,5725.1786,74.7103,-0.0740,0.9368,1.7838,0.0060
lasso,Lasso Regression,64.4014,5787.1680,75.0225,-0.0829,0.9387,1.7835,0.3140
llar,Lasso Least Angle Regression,64.4014,5787.1666,75.0225,-0.0829,0.9387,1.7835,0.0070
br,Bayesian Ridge,65.1814,5809.6445,75.4771,-0.0964,0.9397,1.8172,0.0090
ridge,Ridge Regression,64.7511,5886.9906,75.6342,-0.1003,0.9433,1.7919,0.0110
lr,Linear Regression,64.7621,5889.7848,75.6503,-0.1007,0.9434,1.7921,0.4210
lar,Least Angle Regression,64.7621,5889.7846,75.6503,-0.1007,0.9434,1.7921,0.0070
dummy,Dummy Regressor,65.5708,5766.6740,75.4352,-0.1022,0.9387,1.8249,0.0050
et,Extra Trees Regressor,64.8691,5958.6619,75.9178,-0.1128,0.9403,1.7734,0.0190


In [9]:
# El mejor modelo es el omp asi que voy a usar ese:
omp = create_model('omp')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,81.2098,8063.4636,89.7968,-0.1575,1.0450,1.8786
1,68.5438,6429.1661,80.1821,0.0231,1.0473,3.9535
2,58.1236,4487.4348,66.9883,0.1332,1.2201,0.5755
3,40.2863,2512.5836,50.1257,-0.0235,0.3437,0.2852
4,62.4206,5793.4110,76.1145,-0.0390,1.0118,2.1200
5,71.6245,7375.5287,85.8809,-0.7664,1.4018,4.1417
6,65.4313,6350.5633,79.6904,-0.0306,0.8710,1.2708
7,62.5381,5351.1775,73.1517,-0.0094,0.7136,0.9539
8,66.6399,5523.1353,74.3178,0.0821,0.7944,1.1717


In [10]:
tuned_omp = tune_model(omp)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,82.0237,8452.1037,91.9353,-0.2133,1.0702,1.9834
1,65.8016,5849.8356,76.4842,0.1112,0.9988,3.2516
2,58.2687,4481.4984,66.9440,0.1343,1.2406,0.5623
3,38.8584,2270.4295,47.6490,0.0751,0.3360,0.2879
4,65.3031,6130.2635,78.2960,-0.0995,1.0240,2.1940
5,68.8238,6875.8394,82.9207,-0.6468,1.3696,3.8270
6,67.3014,6514.9319,80.7151,-0.0573,0.8304,1.1633
7,62.1711,5193.6857,72.0672,0.0203,0.7074,0.9423
8,66.1263,5436.0814,73.7298,0.0965,0.7855,1.1311


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [11]:
evaluate_model(tuned_omp)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [12]:
final_omp = finalize_model(tuned_omp)

In [13]:
# Guardo el modelo para usarlo en streamlit

save_model(final_omp, 'city_predictor')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Cost of Living',
                                              'Travel Connectivity', 'Safety',
                                              'Healthcare', 'Education',
                                              'Environmental Quality', 'Economy',
                                              'Leisure & Culture'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('actual_estimator',
                  OrthogonalMatchingPursuit(n_nonzero_coefs=2))]),
 'city_predictor.pkl')